<a href="https://colab.research.google.com/github/AbrahamG20/TFMachineLearning/blob/main/TFMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>